[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/02-dynamic-routes.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/02-dynamic-routes.ipynb)

# Dynamic Routes

In semantic-router there are two types of routes that can be chosen. Both routes belong to the `Route` object, the only difference between them is that _static_ routes return a `Route.name` when chosen, whereas _dynamic_ routes use an LLM call to produce parameter input values.

For example, a _static_ route will tell us if a query is talking about mathematics by returning the route name (which could be `"math"` for example). A _dynamic_ route can generate additional values, so it may decide a query is talking about maths, but it can also generate Python code that we can later execute to answer the user's query, this output may look like `"math", "import math; output = math.sqrt(64)`.

## Installing the Library

In [ ]:
!pip install -qU semantic-router==0.0.15

_**⚠️ If using Google Colab, install the prerequisites and then restart the notebook before continuing**_

## Initializing Routes and RouteLayer

Dynamic routes are treated in the same way as static routes, let's begin by initializing a `RouteLayer` consisting of static routes.

In [1]:
from semantic_router import Route

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

We initialize our `RouteLayer` with our `encoder` and `routes`. We can use popular encoder APIs like `CohereEncoder` and `OpenAIEncoder`, or local alternatives like `FastEmbedEncoder`.

In [2]:
import os
from getpass import getpass
from semantic_router import RouteLayer
from semantic_router.encoders import CohereEncoder, OpenAIEncoder

# dashboard.cohere.ai
# os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
#     "Enter Cohere API Key: "
# )
# platform.openai.com
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

# encoder = CohereEncoder()
encoder = OpenAIEncoder()

rl = RouteLayer(encoder=encoder, routes=routes)

2024-01-07 15:23:12 INFO semantic_router.utils.logger Initializing RouteLayer


We run the solely static routes layer:

In [3]:
rl("how's the weather today?")

RouteChoice(name='chitchat', function_call=None)

## Creating a Dynamic Route

As with static routes, we must create a dynamic route before adding it to our route layer. To make a route dynamic, we need to provide a `function_schema`. The function schema provides instructions on what a function is, so that an LLM can decide how to use it correctly.

In [4]:
from datetime import datetime
from zoneinfo import ZoneInfo


def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London".
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%H:%M")

In [5]:
get_time("America/New_York")

'09:23'

To get the function schema we can use the `get_schema` function from the `function_call` module.

In [6]:
from semantic_router.utils.function_call import get_schema

schema = get_schema(get_time)
schema

{'name': 'get_time',
 'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London".\n:type timezone: str\n:return: The current time in the specified timezone.',
 'signature': '(timezone: str) -> str',
 'output': "<class 'str'>"}

We use this to define our dynamic route:

In [7]:
time_route = Route(
    name="get_time",
    utterances=[
        "what is the time in new york city?",
        "what is the time in london?",
        "I live in Rome, what time is it?",
    ],
    function_schema=schema,
)

Add the new route to our `layer`:

In [8]:
rl.add(time_route)

2024-01-07 15:23:16 INFO semantic_router.utils.logger Adding `get_time` route


Now we can ask our layer a time related question to trigger our new dynamic route.

In [9]:
# https://platform.openai.com/
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

rl("what is the time in new york city?")

2024-01-07 15:23:17 WARNING semantic_router.utils.logger No LLM provided for dynamic route, will use OpenAI LLM default. Ensure API key is set in OPENAI_API_KEY environment variable.
2024-01-07 15:23:17 INFO semantic_router.utils.logger Extracting function input...


RouteChoice(name='get_time', function_call={'timezone': 'new york city'})

---